# Data Science Capstone Project

### Battle of the Neighborhoods: Real Estate Advisor

**_Vadiraj Joshi_**

## **Contents**

* [Introduction/Business Problem](https://github.com/vadiraj-joshi/coursera_capstone_project/blob/master/Coursera_Capstone_Project_BON_Final.ipynb)
* [Data](https://github.com/vadiraj-joshi/coursera_capstone_project/blob/master/Coursera_Capstone_Project_BON_Final.ipynb)
* [Report](https://github.com/vadiraj-joshi/coursera_capstone_project/blob/master/Coursera_Capstone_Project_BON_Final.ipynb)

### Introduction / Business Problem

Location, Location and Location! It is what matters most when a startup is looking up to setup a new office location or restaurateurs or fast food giants like dominos, KFC zooming in for nice spot or an ice cream shop like corner house in busy junction. 

In this project, we will try to build a feature for recommendation engine which might be used by realtor for their web portal and also mobile platform to send personalized recommendations to its subscribers. It could be self trained bot / personal assist with realtor features which specializes in showing personalized recommendations based on individuals preferences as everyone’s requirements are not the same. Let’s illustrate with an use case.

Use Case:-
A student has got an admission in foreign unitversity (CMU, Pittsburg) and now in search of accomodation. He / She will do lot of research and talk to their friends / families for opinion of best neighborhoods in the state. They stumble upon advisor app which understands user preferences and suggests best place to live in. While we could debate which colleges within the are the 'best,' that's not the goal of this case. Instead, ride along with us as we determine which neighborhood in Pennsylvania are the best for students who look for everything at one's reach. You know, the type of places where there's energy and stimulation, but the type of place you can walk around at night. The happiest, chillest, most downright fun places where one can think of going to college where you won't break the bank, or worry about their well being. While some of these may not seem like a 'real' college town per se, when you look at the numbers, it's hard to argue against !

Assumption:- Student already has couple friends / relatives in Allegheny County.

Is it the booming businesses? The good food? The top-rated schools? The diversity?


### Data

Now that we have understood the business requirements. It is time to analyze and gather data for it. We will be using *Pennsylvania municipalties* data for helping our student to find best neighborhood. We have collected data from valid sources, opensocrata for our analysis and performed preliminary analysis for better understanding the data and preparing it for modelling. Once we collected the required data, next step is preparation and understand it better by applying normalization techniques - Getting rid of null values, duplicate rows, data wrangling and formatting the data (Standardization). After the data is standardized the qualified data is ready to be processed. We will query for each of their geo locations using geocoder library and venues from FourSquare API. For few of the locations, foursquare doesn't have data, we shall drop such rows as they are not useful for our analysis.

Based on the amenities provided by foursquare in the neighbourhood, Then cluster the nearby venues to find out the top ten amenities available for each location and extract those features. Based on the amenities cluster the neighbourhoods which have similar characteristics. As we know user preferences, these clusters can be analyzed and recommended for living !

### CODING BEGINS TO EXPLORE DATA 

In [206]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
!pip install geocoder
import geocoder 
!pip install folium
import folium
import geopy
import tqdm
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

*_Lets download the data from data source for analysis_*

In [2]:
!wget -O Pennsylvania_Municipalities.csv https://opendata.socrata.com/api/views/vr4q-nrmd/rows.csv?accessType=DOWNLOAD
print("Data source downloaded")

--2018-10-12 16:27:15--  https://opendata.socrata.com/api/views/vr4q-nrmd/rows.csv?accessType=DOWNLOAD
Resolving opendata.socrata.com (opendata.socrata.com)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to opendata.socrata.com (opendata.socrata.com)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘Pennsylvania_Municipalities.csv’

    [ <=>                                   ] 43,151      --.-K/s   in 0.03s   

Last-modified header invalid -- time-stamp ignored.
2018-10-12 16:27:15 (1.33 MB/s) - ‘Pennsylvania_Municipalities.csv’ saved [43151]

Data source downloaded


In [3]:
df = pd.read_csv("Pennsylvania_Municipalities.csv")
df.head()

,Municipality,Liquor Ban,Beer Ban,Distributor Ban,State Store Ban
0,"Adams Township, Butler County",Allowed,1935,Allowed,Allowed
1,"Adams Township, Snyder County",1955,1955,Allowed,Allowed
2,"Akron Borough, Lancaster County",1953,Allowed,Allowed,Allowed
3,"Aldan Borough, Delaware County",1941,1941,Allowed,Allowed
4,"Alexandria Borough, Huntingdon County",Allowed,1934,Allowed,Allowed


Let's get rid of not required columns that are not useful for our data science project

In [11]:
penn_data = df[["Municipality"]]

In [12]:
penn_data.head()

,Municipality
0,"Adams Township, Butler County"
1,"Adams Township, Snyder County"
2,"Akron Borough, Lancaster County"
3,"Aldan Borough, Delaware County"
4,"Alexandria Borough, Huntingdon County"


In [13]:
penn_data[["Borough", "County"]] = penn_data['Municipality'].str.split(',',expand=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [38]:
penn_data

,Borough,County
0,Adams Township,Butler County
2,Akron Borough,Lancaster County
3,Aldan Borough,Delaware County
4,Alexandria Borough,Huntingdon County
5,Allegheny Township,Butler County
7,Anthony Township,Lycoming County
8,Antrim Township,Franklin County
9,Ararat Township,Susquehanna County
10,Arendtsville Borough,Adams County
11,Armenia Township,Bradford County


Now we have required data, but still need some more work on this data.

In [17]:
penn_data = penn_data[["Borough", "County"]]
penn_data.head()
len(penn_mun)

684

Drop duplicates on the basis of Borough

In [16]:
penn_data.drop_duplicates(subset=['Borough'], inplace=True)

In [25]:
penn_data.shape

(578, 2)

Lets check stats on the table for quick view

In [19]:
penn_data.describe()

,Borough,County
count,578,578
unique,578,55
top,South Franklin Township,York County
freq,1,40


We shall group based on County and for simplicity

In [20]:
penn_data_grp = penn_data.groupby(['County'], sort=False)['Borough'].apply(','.join).reset_index()
penn_data_grp.head()

,County,Borough
0,Butler County,"Adams Township,Allegheny Township,Brady Townsh..."
1,Lancaster County,"Akron Borough,Caernarvon Township,Drumore Town..."
2,Delaware County,"Aldan Borough,Bethel Township,Brookhaven Borou..."
3,Huntingdon County,"Alexandria Borough,Barree Township,Cass Townsh..."
4,Lycoming County,"Anthony Township,Cogan House Township,Gamble T..."


In [21]:
penn_data_grp.describe()

,County,Borough
count,55,55
unique,55,55
top,Fulton County,"Conewango Township,Pine Grove Township,Southwe..."
freq,1,1


In [22]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [23]:
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.02 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.57 MB/s


Now, we are going to use geocoding api to fetch lat long for each row items. 

In [26]:
from geopy.exc import GeocoderTimedOut
from time import sleep
def do_geocode(address):
    geopy = Nominatim()
    try:
        return geopy.geocode(address)
    except GeocoderTimedOut:
        sleep(1)
        return do_geocode(address)


for index, row in penn_data.iterrows():
    try:
        #address_1 = row['Borough'] 
        #address_2 = address_1.split(',')[-1]
        #address = address_2+","+"Pennsylvania,USA"
        address_1 = row['Borough'] 
        address = address_1+","+row['County']+","+"Pennsylvania"+","+"USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        #location = do_geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['District'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
        pass
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address = row['County']+","+"Pennsylvania,USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        #location = do_geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        #print(row['District'],address, latitude, longitude)
        n_hood
        pass

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [27]:
n_hood.head()

,Latitude,Longitude
0,40.717358,-80.012873
1,40.164064,-76.205099
2,39.919412,-75.400164
3,40.344633,-78.028119
4,41.138114,-79.738746


Concatenate the latitude and longitude values to the main dataframe.

In [30]:
penn_data_geo = pd.concat([penn_data, n_hood[['Latitude', 'Longitude']]], axis=1)
penn_data_geo.shape

(657, 4)

In [31]:
penn_data_geo.head()

,Borough,County,Latitude,Longitude
0,Adams Township,Butler County,40.717358,-80.012873
1,NaN,NaN,40.164064,-76.205099
2,Akron Borough,Lancaster County,39.919412,-75.400164
3,Aldan Borough,Delaware County,40.344633,-78.028119
4,Alexandria Borough,Huntingdon County,41.138114,-79.738746


Drop NaN values if any from the data frame

In [32]:
penn_data_geo = penn_data_geo.dropna(subset = ['Borough', 'County', 'Latitude', 'Longitude'])

In [51]:
penn_data_geo.head()

,Borough,County,Latitude,Longitude
0,Adams Township,Butler County,40.717358,-80.012873
2,Akron Borough,Lancaster County,39.919412,-75.400164
3,Aldan Borough,Delaware County,40.344633,-78.028119
4,Alexandria Borough,Huntingdon County,41.138114,-79.738746
5,Allegheny Township,Butler County,41.347269,-77.022035


In [34]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(penn_data_geo['County'].unique()),
        penn_data_geo.shape[0]
    )
)

The dataframe has 55 boroughs and 499 neighborhoods.


After retrieving all necessary data for Pennsylvania. Lets explore using foursquare API Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

To begin with we will visualize all neighboroods on map

In [35]:
address = 'Pennsylvania,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pennsylvania are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Pennsylvania are 40.9699889, -77.7278831.


In [36]:
map_penn = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(penn_data_geo['Latitude'], penn_data_geo['Longitude'], penn_data_geo['County'], penn_data_geo['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_penn) 
    
map_penn

Let's simplify the above map and segment and cluster only the neighborhoods in Allegheny County. So let's slice the original dataframe and create a new dataframe of the Allegheny data.

In [74]:
penn_data_geo = penn_data_geo.reset_index(drop=True)
#penn_data_geo.columns = penn_data_geo.columns.str.lstrip()
penn_data_geo['County'] = penn_data_geo['County'].str.strip()

In [107]:
penn_data.describe()

,Borough,County
count,578,578
unique,578,55
top,South Franklin Township,York County
freq,1,40


### Now lets Explore Neighborhoods in Allegheny County

In [238]:
penn_data_new = penn_data_geo[penn_data_geo.County == 'Allegheny County'].reset_index(drop=True)
len(penn_data_new)
#columns = ["Borough", "County", "Latitude", "Longitude"]
#penn_data_new = penn_data_new.reindex(columns=columns)
penn_data_new.head()

,Borough,County,Latitude,Longitude
0,Bellevue Borough,Allegheny County,40.398036,-76.811517
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951
3,Edgewood Borough,Allegheny County,40.752960,-77.928250
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000


Lets visualize Allegheny County on the map

In [180]:
address = 'Allegheny County, Pennsylvania'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Allegheny County are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Allegheny County are 40.4597204, -79.9760405.


In [181]:
# create map of Manhattan using latitude and longitude values
map_allegheny = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(penn_data_new['Latitude'], penn_data_new['Longitude'], penn_data_new['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_allegheny)  
    
map_allegheny

Looking at the above map, its interesting. Lets check out neighborhoods for best recommendations 

In [184]:
penn_data_new.loc[2, 'Borough']

'Bradford Woods Borough'

Lets explore *_Bradford Woods Borough_*

In [185]:
neighborhood_latitude = penn_data_new.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = penn_data_new.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = penn_data_new.loc[2, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bradford Woods Borough are 39.8766669, -75.3919513045786.


Define Foursquare Credentials and Version

In [186]:
CLIENT_ID = 'HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO' # your Foursquare ID
CLIENT_SECRET = 'T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO
CLIENT_SECRET:T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ


In [187]:
radius = 1000
LIMIT = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HDD0TFIAZXBAGTHQFUMMFD1TXISGGMXQPMJNNYE1SIT5Q1FO&client_secret=T5ZUW2WMHRJ2JDWKN4TOX1JITUJQ4UFOQV4E1U3M0GFJ1HVJ&v=20180924&ll=39.8766669,-75.3919513045786&radius=1000&limit=500'

In [188]:
import requests
import json
from pandas.io.json import json_normalize


In [189]:
results = requests.get(url).json()

In [190]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [191]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
len(nearby_venues)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
len(nearby_venues)
print('{} venues by Foursquare.'.format(nearby_venues.shape[0]))

46 venues by Foursquare.


Based on above results, we get 46 venues for the selected neighborhood. Lets analyze further for nearby one's

In [192]:
#Call this function on each neighborhood and create a new dataframe called allegheny_venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(nearby_venues)
    return(nearby_venues)

In [193]:
allegheny_venues = getNearbyVenues(names=penn_data_new['Borough'],
                                   latitudes=penn_data_new['Latitude'],
                                   longitudes=penn_data_new['Longitude']
                                  )

Bellevue Borough
Ben Avon Borough
Bradford Woods Borough
Edgewood Borough
Forest Hills Borough
Ingram Borough


In [196]:
allegheny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ben Avon Borough,39.848541,-75.486389,Bethel Community Park,39.846563,-75.482629,Park
1,Bradford Woods Borough,39.876667,-75.391951,Firehouse Subs,39.877664,-75.394346,Sandwich Place
2,Bradford Woods Borough,39.876667,-75.391951,Lowe's Home Improvement,39.879045,-75.394831,Hardware Store
3,Bradford Woods Borough,39.876667,-75.391951,Wawa,39.876445,-75.393380,Convenience Store
4,Bradford Woods Borough,39.876667,-75.391951,Starbucks,39.876556,-75.395671,Coffee Shop


Lets find out how many unique categories can be curated from all the returned venues

In [197]:
print('There are {} uniques categories.'.format(len(allegheny_venues['Venue Category'].unique())))
allegheny_venues.groupby('Neighborhood').count()

There are 25 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ben Avon Borough,1,1,1,1,1,1
Bradford Woods Borough,28,28,28,28,28,28
Forest Hills Borough,1,1,1,1,1,1
Ingram Borough,3,3,3,3,3,3


### Analyze Each Neighborhood

In [198]:
# one hot encoding
allegheny_venues_onehot = pd.get_dummies(allegheny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
allegheny_venues_onehot['Neighborhood'] = allegheny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [allegheny_venues_onehot.columns[-1]] + list(allegheny_venues_onehot.columns[:-1])
allegheny_venues_onehot = allegheny_venues_onehot[fixed_columns]

allegheny_venues_onehot.head()

,Neighborhood,BBQ Joint,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega,Diner,...,Liquor Store,Mobile Phone Shop,Motel,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Sandwich Place,Shopping Plaza
0,Ben Avon Borough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Bradford Woods Borough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Bradford Woods Borough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bradford Woods Borough,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bradford Woods Borough,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [199]:
allegheny_venues_onehot.shape

(33, 26)

In [200]:
allegheny_venues_grouped = allegheny_venues_onehot.groupby('Neighborhood').mean().reset_index()
allegheny_venues_grouped

,Neighborhood,BBQ Joint,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega,Diner,...,Liquor Store,Mobile Phone Shop,Motel,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Sandwich Place,Shopping Plaza
0,Ben Avon Borough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bradford Woods Borough,0.000000,0.035714,0.000000,0.035714,0.035714,0.035714,0.035714,0.0,0.071429,...,0.142857,0.035714,0.000000,0.035714,0.035714,0.035714,0.071429,0.035714,0.035714,0.071429
2,Forest Hills Borough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ingram Borough,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Lets print each neighborhood along with the top 5 most common venues

In [201]:
num_top_venues = 5

for hood in allegheny_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = allegheny_venues_grouped[allegheny_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ben Avon Borough----
                venue  freq
0                Park   1.0
1           BBQ Joint   0.0
2      Hardware Store   0.0
3      Sandwich Place   0.0
4  Salon / Barbershop   0.0


----Bradford Woods Borough----
            venue  freq
0    Liquor Store  0.14
1  Shopping Plaza  0.07
2     Pizza Place  0.07
3           Diner  0.07
4   Grocery Store  0.07


----Forest Hills Borough----
                venue  freq
0       Deli / Bodega   1.0
1           BBQ Joint   0.0
2      Hardware Store   0.0
3      Sandwich Place   0.0
4  Salon / Barbershop   0.0


----Ingram Borough----
            venue  freq
0       BBQ Joint  0.33
1         Brewery  0.33
2           Motel  0.33
3  Hardware Store  0.00
4  Sandwich Place  0.00




In [202]:
#Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

New dataframe and display the top 10 venues for each neighborhood.

In [203]:
#function to display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = allegheny_venues_grouped['Neighborhood']

for ind in np.arange(allegheny_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(allegheny_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

4

In [204]:
allegheny_venues_grouped.head()

,Neighborhood,BBQ Joint,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega,Diner,...,Liquor Store,Mobile Phone Shop,Motel,Optical Shop,Park,Pet Store,Pizza Place,Salon / Barbershop,Sandwich Place,Shopping Plaza
0,Ben Avon Borough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bradford Woods Borough,0.000000,0.035714,0.000000,0.035714,0.035714,0.035714,0.035714,0.0,0.071429,...,0.142857,0.035714,0.000000,0.035714,0.035714,0.035714,0.071429,0.035714,0.035714,0.071429
2,Forest Hills Borough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ingram Borough,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Cluster Neighborhoods

In [208]:
# set number of clusters
kclusters = 4

allegheny_grouped_clustering = allegheny_venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(allegheny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 2, 1], dtype=int32)

In [212]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ben Avon Borough,Park,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega
1,Bradford Woods Borough,Liquor Store,Shopping Plaza,Grocery Store,Pizza Place,Diner,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store
2,Forest Hills Borough,Deli / Bodega,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Diner
3,Ingram Borough,BBQ Joint,Brewery,Motel,Grocery Store,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega


In [220]:
penn_data_new

,Borough,County,Latitude,Longitude
0,Bellevue Borough,Allegheny County,40.398036,-76.811517
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951
3,Edgewood Borough,Allegheny County,40.752960,-77.928250
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000
5,Ingram Borough,Allegheny County,40.080670,-76.241128


In [221]:
penn_data_new_1 = penn_data_new.iloc[[1,2,4,5],:]

Run k-means algorithm to cluster the neighborhood into 4 clusters and label them

In [228]:
penn_data_merged = penn_data_new_1

# add clustering labels
penn_data_merged['Cluster Labels'] = kmeans.labels_

# merge penn_grouped with penn_data to add latitude/longitude for each neighborhood
penn_data_merged = penn_data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

penn_data_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ben Avon Borough,Allegheny County,39.848541,-75.486389,0,Park,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega
2,Bradford Woods Borough,Allegheny County,39.876667,-75.391951,3,Liquor Store,Shopping Plaza,Grocery Store,Pizza Place,Diner,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store
4,Forest Hills Borough,Allegheny County,40.691662,-80.371000,2,Deli / Bodega,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Diner
5,Ingram Borough,Allegheny County,40.080670,-76.241128,1,BBQ Joint,Brewery,Motel,Grocery Store,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega


Finally, lets visualize the resulting clusters on the map

In [232]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(penn_data_merged['Latitude'].round(4), penn_data_merged['Longitude'].round(4), penn_data_merged['Borough'], penn_data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Lets Examine Clusters

### *_Cluster 1 - Place to relax and eatout - Main Junction_*

In [245]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 0, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Ben Avon Borough,Park,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega


### *_Cluster 2 - Seems to be tourist spot with restaurants, pubs and motels_*

In [246]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 1, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Ingram Borough,BBQ Joint,Brewery,Motel,Grocery Store,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Deli / Bodega


### *_Cluster 3 - Shopping hub with lot of Malls, Stores and Bank_*

In [247]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 2, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Forest Hills Borough,Deli / Bodega,Shopping Plaza,Grocery Store,Bank,Brewery,Burger Joint,Bus Station,Coffee Shop,Convenience Store,Diner


### *_Cluster 4 - Looks like happening place with everything_*

In [248]:
penn_data_merged.loc[penn_data_merged['Cluster Labels'] == 3, penn_data_merged.columns[[0] + list(range(5, penn_data_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bradford Woods Borough,Liquor Store,Shopping Plaza,Grocery Store,Pizza Place,Diner,Bank,Burger Joint,Bus Station,Coffee Shop,Convenience Store


## Report

### *_Methodology_*:-

The methodology I used is pretty simple. As illustrated above, I used K-Means clusttering algorithm to cluster based on the amenities, cluster the neighbourhoods which have similar characteristics. As we know user preferences are taken into consideration such as Banks, Shopping Malls, Bus Station, Grocery, Resturants, Coffee Shops, Deli Foods/ Bakeries etc ...
Find out which ones are best for living.  Seems to be simple question, definitely can be done in much better ways and options are debatable and that's why we called it BON :)

### *_Results_*:-

Based on the amenities available in each area, the city-county combinations are divided into 4 clusters. 
Each cluster has a unique amenity.
For example Cluster 1 is group of amenities found in any key junction of neighborhood with parks, shopping, grocery, bank
Cluster 2 is the group which contains needs for tourist ppl such as restaurants and motels for accomodation
Cluster 3 is mostly about food joints and super markets
Cluster 4 is the grouping of most happening places consisting of restaurants, nightlife, pubs, malls, banks & entertainment centers


### *_Discussion_*:-

Without a lot more work in the initial data exploration and methodology phase, it would not be possible to figure out what are top amenities in the neighborhood that helps in making the decision for living in that area !

Though the data has gone through exploratory analysis, some of the issues can be found during actual run of data. For ex: few locations didn't return geo location and few others didn't return any result for FourSquare API and couldn't find nearby venues. My observation is that the Data Science methodology is a highly iterative process which needs going back and forth to tune the data as needed. 


### *_Conclusion_*:-

After examination of the clusters we discovered that '_Forest Hills Borough_' in Allegheny County is closest match and suitable option for recommendation and will be shown as top priority. 

Note:- Assuming that one of the family lives in Forest hills it would be easier for the student to use their car for college commuting (sometimes) otherwise Bus station is also available. Also Forest hills is affordable and at the same time has numerous part time job vacancies for tutoring to small kids during free time.

---

4 	Forest Hills Borough 	Deli / Bodega 	Shopping Plaza 	Grocery Store 	Bank 	Brewery 	Burger Joint 	Bus Station 	Coffee Shop 	Convenience Store 	Diner

In fact, it is quite likely I will return to this idea and build upon it for nearby recommendations using Alexa with Voice !

Thanks for reading.